# ヒートマップ

ヒートマップをプロットするための基本的な考え方は、特定のイベントの確率に関するフィードバックを視覚的に取得することである。

範囲0〜1の確率をとり、範囲（255、0、 0） - （0、0、255）またはOpencvはカラーマップを提供している。 COLORMAP _JETを使用することに興味があるかもしれません。

そして今、あなたは0-1の代わりに0-255の範囲で確率を正規化しなければなりません、そしてあなたは希望する出力を得るためにcv2.applyColorMap（input_ prob、cv2.COLORMAP_JET）を使うかもしれません。

# Simple-Python-heatmap

[link](https://github.com/Muhamob/Simple-Python-heatmap)

In [5]:
import numpy as np
import cv2

start = 1
duration = 10
fps = '30'
cap = cv2.VideoCapture(0)
outfile = 'heatmap.mp4'


while True:
    try:
        _, f = cap.read()
        f = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
        f = cv2.GaussianBlur(f, (11, 11), 2, 2)
        cnt = 0
        res = 0.05*f
        res = res.astype(np.float64)
        break
    except:

fgbg = cv2.createBackgroundSubtractorMOG2(history=1, varThreshold=100,
                                          detectShadows=True)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (13, 13))
cnt = 0
sec = 0
while True:

    cnt += 1
    if cnt % int(fps) == 0:
        print(sec)
        sec += 1
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame, None, 0.01)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 2, 2)
    gray = gray.astype(np.float64)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
    fgmask = fgmask.astype(np.float64)
    res += (40 * fgmask + gray) * 0.01
    res_show = res / res.max()
    res_show = np.floor(res_show * 255)
    res_show = res_show.astype(np.uint8)
    res_show = cv2.applyColorMap(res_show, cv2.COLORMAP_JET)
    cv2.imshow('s', res_show)
    out.write(frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()

out.release()
cv2.destroyAllWindows()

0
1
2


# 物体追跡

## OpenCVでの密なオプティカルフロー

Lucas-Kanade法が出力するオプティカルフローは疎な特徴である(上記のコードではShi-Tomasiアルゴリズムによってコーナーを検出している)。

OpenCVは密なオプティカルフローを検出するためのアルゴリズムを別に用意している。画像中の全画素に対してオプティカルフローを計算する。Gunner Farnebackが2003年に発表した “Two-Frame Motion Estimation Based on Polynomial Expansion” で提案されたアルゴリズムに基づいている。

以下のサンプルはこのアルゴリズムを使って密なオプティカルフローを計算している。出力として，オプティカルフローベクトル (u,v) を格納した2チャンネルの配列が返ってくる。オプティカルフローの強度と方向を計算し、カラーコード化した画像を下に示す。
方向はHue成分，強度はValue成分によって表わしている。

In [2]:
import cv2
import numpy as np
cap = cv2.VideoCapture('vtest.avi')

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(
        prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
#     hsv[..., 0] = ang*180/np.pi/2
#     hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
#     rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # 0を除く最小値の1.5倍まで0にする
    res_show = mag / mag.max()
    res_show = np.floor(res_show * 255)
    res_show = res_show.astype(np.uint8)
    res_show = cv2.applyColorMap(res_show, cv2.COLORMAP_JET)

    alpha = 0.6
    res_show = cv2.addWeighted(frame2, alpha, res_show, 1 - alpha, 0)

    cv2.imshow('blend', res_show)
#     cv2.imshow('frame2', frame2)
    k = cv2.waitKey(79) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png', frame2)
        cv2.imwrite('opticalhsv.png', rgb)
    prvs = next

cap.release()
cv2.destroyAllWindows()

# サンプルコード

## QT douga

In [1]:
import sys
from PyQt5 import QtCore, QtGui, QtWidgets
import cv2
import os, zipfile, datetime
now = datetime.datetime.now()
dirname = 'test_{0:%Y%m%d}'.format(now)

class QtCapture(QtWidgets.QWidget):
    def __init__(self, *args):
        super(QtWidgets.QWidget, self).__init__()

        self.fps = 24
        self.cap = cv2.VideoCapture(*args)

        self.video_frame = QtWidgets.QLabel()
        lay = QtWidgets.QVBoxLayout()
        lay.setContentsMargins(0,0,0,0)
        lay.addWidget(self.video_frame)
        self.setLayout(lay)

        # ------ Modification ------ #
        self.isCapturing = False
        self.ith_frame = 1
        # ------ Modification ------ #

    def setFPS(self, fps):
        self.fps = fps

    def nextFrameSlot(self):
        ret, frame = self.cap.read()

        # ------ Modification ------ #
        # Save images if isCapturing
        if self.isCapturing:
            if not(os.path.isdir(dirname)):
                os.mkdir(dirname)
            cv2.imwrite('./'+dirname+'/img_%05d.jpg'%self.ith_frame, frame)
            self.ith_frame += 1
        # ------ Modification ------ #

        # My webcam yields frames in BGR format
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = QtGui.QImage(frame, frame.shape[1], frame.shape[0], QtGui.QImage.Format_RGB888)
        pix = QtGui.QPixmap.fromImage(img)
        self.video_frame.setPixmap(pix)

    def start(self):
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.nextFrameSlot)
        self.timer.start(1000./self.fps)

    def stop(self):
        self.timer.stop()

    # ------ Modification ------ #
    def capture(self):
        if not self.isCapturing:
            self.isCapturing = True
        else:
            self.isCapturing = False
    # ------ Modification ------ #

    def deleteLater(self):
        self.cap.release()
        super(QtWidgets.QWidget, self).deleteLater()


class ControlWindow(QtWidgets.QWidget):
    def __init__(self):
        QtWidgets.QWidget.__init__(self)
        self.capture = None

        self.start_button = QtWidgets.QPushButton('Start')
        self.start_button.clicked.connect(self.startCapture)
        self.quit_button = QtWidgets.QPushButton('End')
        self.quit_button.clicked.connect(self.endCapture)
        self.end_button = QtWidgets.QPushButton('Stop')

        # ------ Modification ------ #
        self.capture_button = QtWidgets.QPushButton('Capture')
        self.capture_button.clicked.connect(self.saveCapture)
        # ------ Modification ------ #

        vbox = QtWidgets.QVBoxLayout(self)
        vbox.addWidget(self.start_button)
        vbox.addWidget(self.end_button)
        vbox.addWidget(self.quit_button)

        # ------ Modification ------ #
        vbox.addWidget(self.capture_button)
        # ------ Modification ------ #

        self.setLayout(vbox)
        self.setWindowTitle('Control Panel')
        self.setGeometry(100,100,200,200)
        self.show()

    def startCapture(self):
        if not self.capture:
            self.capture = QtCapture(0)
            self.end_button.clicked.connect(self.capture.stop)
            # self.capture.setFPS(1)
            self.capture.setParent(self)
            self.capture.setWindowFlags(QtCore.Qt.Tool)
        self.capture.start()
        self.capture.show()

    def endCapture(self):
        self.capture.deleteLater()
        self.capture = None

    # ------ Modification ------ #
    def saveCapture(self):
        if self.capture:
            self.capture.capture()
    # ------ Modification ------ #



if __name__ == '__main__':

    import sys
    app = QtWidgets.QApplication(sys.argv)
    window = ControlWindow()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\shinkawa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# その他

## Anacondaにプロキシ設定をする

Anacondaのルートフォルダに.condarcファイルを作成する。

- ルートフォルダ環境によって異なるが、下記のいずれになります。
 - C:\Users[ユーザー名]\Anaconda3
 - C:\Program Files\Anaconda3
 - C:\Users[ユーザー名]\AppData\Local\Continuum\anaconda3

エクスプローラーから直接「.condarc」ファイルを作成できないので、 
先に「condarc」を作成し、コマンドラインでファイル名を.condarc変更する。

ren condarc .condarc

- .condarcをお好みのテキストエディタで編集し、プロキシの設定を追記します。  

    proxy_servers: http: http://[プロキシアドレス]:[ポート番号] https: https://[プロキシアドレス]:[ポート番号] 
    

- コマンドラインから以下のコマンドで設定内容を確認できます。  

   conda config --show  
   

- proxy_serversのところに正しいプロキシであれば設定完了です。